# RL -- Projet "Trading automatique"

Ce notebook contient du code de base et quelques explications pour vous aider sur ce sujet.

Vous êtes libres de réaliser ce projet avec des scripts Python ou des Jupyter Notebooks, à votre convenance.

Vous devez télécharger les paquets Python suivants :

```sh
pip install gymnasium
pip install pandas
pip install gym-trading-env-continuous
```

Vous utiliserez l'environnement `gym-trading-env-continuous`, qui est un *fork* de [Gym Trading Env](https://gym-trading-env.readthedocs.io/en/latest/index.html). La différence majeure est expliquée dans ce document ; la documentation originelle reste utilisable.

In [1]:
import numpy as np
import pandas as pd
import gymnasium as gym
import gym_trading_env

## Utilisation des données de simulation

Les données sont dans un format binaire (Pickle) que vous pouvez lire avec Pandas. Vous devez vous assurer que les données sont triées par date.

Des étapes de prétraitement peuvent aider votre apprentissage, par exemple, supprimer les doublons, etc.

In [2]:
def preprocess(df):
    df = df.sort_index()
    df = df.dropna()
    df = df.drop_duplicates()
    return df

df = preprocess(pd.read_pickle('./data/binance-ETHUSD-1h.pkl'))
df.head(5)

,open,high,low,close,volume,date_close
date_open,,,,,,
2020-08-18 07:00:00,430.00,435.00,410.00,430.30,487.154463,2020-08-18 08:00:00
2020-08-18 08:00:00,430.27,431.79,430.27,430.80,454.176153,2020-08-18 09:00:00
2020-08-18 09:00:00,430.86,431.13,428.71,429.35,1183.710884,2020-08-18 10:00:00
2020-08-18 10:00:00,429.75,432.69,428.59,431.90,1686.183227,2020-08-18 11:00:00
2020-08-18 11:00:00,432.09,432.89,426.99,427.45,1980.692724,2020-08-18 12:00:00


### Ajout de *features*

Vous pouvez également rajouter de nouvelles données au DataFrame pour créer de nouvelles *features* que l'agent pourra utiliser.
Voir pour cela la [doc](https://gym-trading-env.readthedocs.io/en/latest/features.html).

Chaque nouvelle *feature* doit commencer par `feature_` pour être détectée.

In [3]:
def preprocess(df):
    df = df.sort_index()
    df = df.dropna()
    df = df.drop_duplicates()

    df['feature_close'] = (df['close'] - df['close'].mean()) / df['close'].std()

    return df

df = preprocess(pd.read_pickle('./data/binance-ETHUSD-1h.pkl'))
df.head(5)

,open,high,low,close,volume,date_close,feature_close
date_open,,,,,,,
2020-08-18 07:00:00,430.00,435.00,410.00,430.30,487.154463,2020-08-18 08:00:00,-1.891634
2020-08-18 08:00:00,430.27,431.79,430.27,430.80,454.176153,2020-08-18 09:00:00,-1.891128
2020-08-18 09:00:00,430.86,431.13,428.71,429.35,1183.710884,2020-08-18 10:00:00,-1.892594
2020-08-18 10:00:00,429.75,432.69,428.59,431.90,1686.183227,2020-08-18 11:00:00,-1.890016
2020-08-18 11:00:00,432.09,432.89,426.99,427.45,1980.692724,2020-08-18 12:00:00,-1.894514


Par défaut, l'agent ne reçoit comme *features* que sa dernière *position* (voir le paragraphe suivant), ce qui ne sera certainement pas suffisant ! À vous d'ajouter les *features* qui seront pertinentes pour que l'agent apprenne la politique optimale...

## Fonctionnement des actions

Une action est une **position**, c'est-à-dire un ratio entre la proportion d'*assets* (exemple : ETH) et la proportion de *fiat* (exemple : USD) dans le portefeuille.
Ainsi, la position `0.5` consiste à avoir exactement 50% d'ETH et 50% d'USD (en vendant l'un ou l'autre pour arriver à ce ratio). `0.1` consiste à avoir 10% d'ETH et 90% d'USD.

Il existe des positions un peu plus complexes :

- `< 0` : une position inférieure à 0 va vendre encore plus d'ETH que le portefeuille n'en contient, pour obtenir des USD. Cela nécessite un emprunt, qui sera remboursé avec un intérêt.
- `> 1` : une position supérieure à 1 va dépenser encore plus d'USD que le portefeuille n'en contient, pour acheter des ETH. Cela nécessite également un emprunt.

Ces positions (qui sont appelées *short* et *margin* en finance) peuvent faire gagner beaucoup à votre agent, mais démultiplient les risques également. Si votre agent fait une bonne affaire, vous pouvez vendre à un prix élevé, racheter quand le prix est plus faible, et rembourser l'emprunt en empochant la différence. En revanche, si votre agent fait une mauvaise affaire, et doit vider son portefeuille pour rembourser l'emprunt, vous perdez automatiquement (`terminated=True`).

### Actions continues

Par rapport à l'environnement `gym-trading-env` d'origine, la version que je vous fournis permet de spécifier directement une position comme action, c'est-à-dire un nombre flottant. Votre agent a donc un contrôle précis sur la position désirée. Cela rajoute de la flexibilité mais rend l'apprentissage beaucoup plus difficile.

Exemple :

In [4]:
env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="data/*.pkl",
    preprocess=preprocess,
    portfolio_initial_value=1_000,
    trading_fees=0.1/100,
    borrow_interest_rate=0.02/100/24,
)

obs, _ = env.reset()
# On veut une position de 88% ETH / 12% USD
obs, reward, terminated, truncated, info = env.step(0.88)
print(obs)
print(info)

[-1.1176176   0.88        0.86199826]
{'idx': 1, 'step': 1, 'date': np.datetime64('2021-01-29T13:00:00.000000000'), 'position_index': None, 'position': 0.88, 'real_position': np.float64(0.8619982420158219), 'data_volume': 47022463.7, 'data_open': 0.05181, 'data_date_close': Timestamp('2021-01-29 14:00:00'), 'data_high': 0.05699, 'data_close': 0.04413, 'data_low': 0.042, 'portfolio_valuation': np.float64(868.8994655314681), 'portfolio_distribution_asset': np.float64(16972.35014223006), 'portfolio_distribution_fiat': np.float64(119.90965375485541), 'portfolio_distribution_borrowed_asset': 0, 'portfolio_distribution_borrowed_fiat': 0, 'portfolio_distribution_interest_asset': 0.0, 'portfolio_distribution_interest_fiat': 0.0, 'reward': np.float64(-0.14052785024653625)}


Par défaut, l'espace des actions est limité à $[-1, 2]$ pour que votre agent ne puisse emprunter que jusqu'à 100%. Vous pouvez empêcher votre agent de prendre de telles positions, ou limiter le risque, en contrôlant les bornes autorisées des actions.

Par exemple, en clippant l'action dans l'intervalle $[0,1]$, vous empêchez l'agent de faire des emprunts.

À l'inverse, vous pouvez augmenter l'intervalle pour permettre des emprunts plus risqués, mais qui peuvent rapporter plus. À vous de choisir !

Vous pouvez changer les bornes via le paramètre `position_range` du constructeur :

In [5]:
env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="data/*.pkl",
    preprocess=preprocess,
    position_range=(0, 1),  # ICI : (borne min, borne max)
    portfolio_initial_value=1_000,
    trading_fees=0.1/100,
    borrow_interest_rate=0.02/100/24,
)

Vous pouvez aussi modifier l'action en sortie de votre algorithme d'apprentissage, de la manière que vous souhaitez (clipping, interpolation, etc.).

### Actions discrètes

Pour simplifier l'apprentissage, vous pouvez utiliser le *wrapper* `gym_trading_env.wrapper.DiscreteActionsWrapper` que je vous fournis, et qui permet de revenir au fonctionnement d'origine de l'environnement `gym-trading-env`. Vous devrez alors spécifier l'ensemble des positions possibles, puis votre agent choisira une position parmi cette liste à chaque pas de temps.
Par exemple, si la liste des positions est `[0, 0.5, 1]` et que l'action choisie est `1`, cela veut dire qu'on veut la position qui correspond au 2e élément de la liste, soit `0.5` (50%/50%).

Vous pouvez rajouter autant d'actions que vous voulez, par exemple `[0, 0.25, 0.5, 1]` ou encore tous les 0.1 entre 0 et 1, etc. Plus il y a d'actions possibles, plus votre agent aura de choix (flexibilité), donc plus son comportement pourra être complexe, mais cela rajoute de la difficulté durant l'entraînement.

N'oubliez pas que vous pouvez autoriser les positions avec emprunt en ajoutant des nombres inférieurs à 0 ou supérieurs à 1 à la liste autorisée.

Exemple :

In [6]:
from gym_trading_env.wrapper import DiscreteActionsWrapper

# Vous pouvez aussi appeler le wrapper `env` pour faire plus simple
# Ici, je fais explicitement la distinction entre `wrapper` et `env`
wrapper = DiscreteActionsWrapper(env, positions=[-1, 0, 0.25, 0.5, 0.75, 1, 2])
obs, _ = wrapper.reset()
# On veut une position de 25% ETH / 75% USD ; cela correspond à la position
# d'index 2 dans la liste ci-dessus
obs, reward, terminated, truncated, info = wrapper.step(2)
print(obs)
print(info)

[-0.16626613  0.25        0.25014377]
{'idx': 1, 'step': 1, 'date': np.datetime64('2023-11-21T14:00:00.000000000'), 'position_index': 2, 'position': 0.25, 'real_position': np.float64(0.2501437786412375), 'data_volume': 0, 'data_open': 1.0949305295944214, 'data_date_close': Timestamp('2023-11-21 15:00:00'), 'data_high': 1.0961307287216187, 'data_close': 1.0955302715301514, 'data_low': 1.0948106050491333, 'portfolio_valuation': np.float64(1000.0093399728391), 'portfolio_distribution_asset': np.float64(228.33336647827292), 'portfolio_distribution_fiat': np.float64(749.8632249955033), 'portfolio_distribution_borrowed_asset': 0, 'portfolio_distribution_borrowed_fiat': 0, 'portfolio_distribution_interest_asset': 0.0, 'portfolio_distribution_interest_fiat': 0.0, 'reward': np.float64(9.339929221864947e-06)}


Notez que, quand les actions continues sont utilisées, la variable `position_index` du dictionnaire `info` n'est pas disponible (c'est logique).

## Changement de la fonction de récompense

Vous pouvez changer la fonction de récompense pour améliorer l'apprentissage de l'agent.
Dans tous les cas, vous serez évalué(e)s sur la valuation du portefeuille à la fin de l'épisode (voir [ci-dessous](#évaluation)), mais cette simple mesure n'est peut-être pas la meilleure fonction de récompense.
D'autres fonctions peuvent encourager l'agent à mieux apprendre, en explorant diverses possibilités, etc.

In [7]:
def reward_function(history):
    return history['portfolio_valuation', -1]

env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="data/*.pkl",
    preprocess=preprocess,
    portfolio_initial_value=1_000,
    trading_fees=0.1/100,
    borrow_interest_rate=0.02/100/24,
    # On spécifie la fonction de récompense
    reward_function=reward_function,
)

## Déroulément d'un épisode

Un épisode se déroule jusqu'à ce que :

- l'agent atteigne la fin des données d'entraînement (nous n'avons plus de nouvelle donnée) => `truncated=True`

- la valeur du portefeuille atteint 0 (l'agent a perdu tout l'argent) => `terminated=True`

Vous devrez probablement entraîner l'agent sur plusieurs épisodes avant que son comportement ne converge.

Pour éviter de sur-apprendre (*overfit*), vous devrez utiliser plusieurs jeux de données via [MultiDatasetTradingEnv](https://gym-trading-env.readthedocs.io/en/latest/multi_datasets.html).

Dans ce cas, chaque épisode utilisera un jeu de données différent (en bouclant si vous demandez plus d'épisodes qu'il n'y a de jeux de données). Vous pouvez accéder au nom du jeu de données de l'épisode en cours via `env.name`.

In [8]:
nb_episodes = 2
for episode in range(1, nb_episodes + 1):
    obs, _ = env.reset()
    print(f'Episode n˚{episode} -- Jeu de donnée {env.name}')
    done = False

    while not done:
        action = env.action_space.sample()
        obs, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

    if terminated:
        print('Argent perdu')
    elif truncated:
        print('Épisode terminé')


Episode n˚1 -- Jeu de donnée yfinance-EURUSD-1h.pkl
Market Return :  5.27%   |   Portfolio Return : -100.00%   |   
Épisode terminé
Episode n˚2 -- Jeu de donnée yfinance-AAPL-1h.pkl
Market Return : 39.94%   |   Portfolio Return : -96.92%   |   
Épisode terminé


## Évaluation

Afin de disposer d'un critère simple pour comparer les différentes solutions, nous utiliserons la valeur du portefeuille (`portfolio_valuation`).
C'est assez simple : on veut que l'agent ait gagné le plus d'argent à la fin de la simulation.

Vous pouvez ajouter ce critère à la liste des métriques affichées à la fin de chaque épisode, pour que ce soit plus visible :

In [9]:
def metric_portfolio_valuation(history):
    return round(history['portfolio_valuation', -1], 2)

env.add_metric('Portfolio Valuation', metric_portfolio_valuation)

done = False
obs, _ = env.reset()

while not done:
    action = env.action_space.sample()
    obs, reward, terminated, truncated, _ = env.step(action)
    done = terminated or truncated

Market Return : 10.54%   |   Portfolio Return : -98.67%   |   Portfolio Valuation : 13.33   |   


Puisque l'environnement peut se dérouler sur plusieurs épisodes (1 par jeu de données), vous devrez calculer la **moyenne des `portfolio_valuation`** sur l'ensemble des jeux de données possibles.

⚠️ Pour que ce soit honnête, vous **devez initialiser l'environnement avec les contraintes** imposées dans le sujet :

- une valeur initiale du portefeuille de `1000` ;
- des frais de 0.1% par transaction ;
- un taux d'intérêt de 0.02% par jour soit 0.02/100/24 par heure.

Sinon, il est beaucoup plus simple d'augmenter la valeur finale...

```py
env = gym.make(
    "MultiDatasetTradingEnv",
    dataset_dir="data/*.pkl",
    preprocess=preprocess,
    # LIGNES SUIVANTES :
    # Valeur initiale du portefeuille
    portfolio_initial_value=1_000,
    # Frais de transactions
    trading_fees=0.1/100,
    # Intérêts sur les prêts
    borrow_interest_rate=0.02/100/24,
)
```

Vous pouvez également accéder à la métrique de `portfolio_valuation` à la fin d'une simulation, si vous voulez par exemple l'ajouter à votre *run* WandB :

In [10]:
portfolio_valuation = env.historical_info['portfolio_valuation', -1]
# Si on avait WandB :
# run.summary['portfolio_valuation'] = portfolio_valuation
# On simule ça par un simple print...
print(portfolio_valuation)

13.32516244673074


Ou bien, pour récupérer les métriques calculées par l'environnement (cela peut être utile pour les ajouter à WandB) :

In [11]:
metrics = env.get_metrics()
print(metrics)
portfolio_valuation = metrics['Portfolio Valuation']
print(portfolio_valuation)

{'Market Return': '10.54%', 'Portfolio Return': '-98.67%', 'Portfolio Valuation': np.float64(13.33)}
13.33


## Conseils

À part les quelques contraintes présentées dans ce fichier (et rappelées sur la page du projet), vous êtes assez libres !

Votre algorithme de RL peut être arbitrairement simple ou complexe. Je liste ici quelques conseils ou pistes, que vous pouvez explorer :

- *Features* : Par défaut, votre agent n'utilise que le prix de l'*asset* (`close`) comme *feature* pour la prise de décision. Vous pouvez ajouter les *features* que vous voulez. En particulier, des métriques spécifiques à la finance peuvent être intéressantes, par exemple pour déterminer le risque que le prix change brutalement (à la hausse ou à la baisse)...

- Algorithme : Vous pouvez utiliser des algorithmes existants, ou en inventer un nouveau. N'hésitez pas à ré-utiliser tout ce que vous avez appris en *Machine Learning* et *Deep Learning*. Typiquement, les données financières sont des données temporelles : certains réseaux de neurones sont plus appropriés que d'autres pour ce genre de tâche...

- Configuration de l'environnement : L'environnement est très extensible ! Vous pouvez par exemple ajouter des *features* dynamiques (pas seulement calculées lors du prétraitement). La [documentation](https://gym-trading-env.readthedocs.io/en/latest/index.html) est très claire et très complète.

Vous pouvez vous inspirer de travaux existants trouvés sur l'Internet à condition de **citer votre source**. Utiliser le travail de quelqu'un d'autre sans le citer sera considéré comme du plagiat.

In [24]:
import numpy as np
import pandas as pd
import gymnasium as gym
import gym_trading_env
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import BaseCallback, CallbackList
import wandb
from wandb.integration.sb3 import WandbCallback
import matplotlib.pyplot as plt
import os
from stable_baselines3.common.utils import get_latest_run_id


## ----------------------------------------------------------------------
## A. FONCTIONS DE PRÉTRAITEMENT ET DE RÉCOMPENSE
## ----------------------------------------------------------------------

def preprocess_v2(df):
    df = df.sort_index().dropna().drop_duplicates()
    # Log Returns
    df["feature_log_returns"] = np.log(df["close"]).diff()
    # Indicateurs de Volatilité (ATR simplifié)
    df['tr1'] = df['high'] - df['low']
    df['tr2'] = np.abs(df['high'] - df['close'].shift(1))
    df['tr3'] = np.abs(df['low'] - df['close'].shift(1))
    df['tr'] = df[['tr1', 'tr2', 'tr3']].max(axis=1)
    df['feature_atr'] = df['tr'].rolling(window=14).mean() / df["close"]
    # Indicateurs de Tendance (MACD)
    ema_fast = df['close'].ewm(span=12, adjust=False).mean()
    ema_slow = df['close'].ewm(span=26, adjust=False).mean()
    df['feature_macd'] = ema_fast - ema_slow
    df['feature_macd_signal'] = df['feature_macd'].ewm(span=9, adjust=False).mean()
    # Indicateurs de Momentum (RSI)
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    # Normalisation finale
    df['feature_rsi'] = 100 - (100 / (1 + rs)) / 100
    df = df.dropna()
    cols_to_normalize = ['feature_log_returns', 'feature_macd', 'feature_macd_signal', 'feature_atr']
    for col in cols_to_normalize:
        if df[col].std() > 0:
            df[col] = (df[col] - df[col].mean()) / df[col].std()
        else:
             df[col] = 0.0
    return df

def reward_function_v2(history):
    # Log-Return différentiel
    prev_val = history['portfolio_valuation', -2]
    curr_val = history['portfolio_valuation', -1]
    if prev_val == 0: return 0
    reward = np.log(curr_val / prev_val)
    return reward

## ----------------------------------------------------------------------
## B. CUSTOM CALLBACK WANDB (Métriques financières)
## ----------------------------------------------------------------------

class CustomTradingCallback(BaseCallback):
    def __init__(self, verbose: int = 0):
        super().__init__(verbose)
        self.episode_num = 0

    def _on_step(self) -> bool:
        if self.locals['dones'][0]:
            self.episode_num += 1
            raw_env = self.training_env.envs[0].unwrapped
            metrics = raw_env.get_metrics()

            # Récupération des retours pour le calcul de performance
            market_return_str = metrics.get('Market Return', '0.00%').strip()
            market_return = float(market_return_str.strip('%')) / 100
            portfolio_return_str = metrics.get('Portfolio Return', '0.00%').strip()
            portfolio_return = float(portfolio_return_str.strip('%')) / 100

            if self.logger is not None:
                self.logger.record("episode/final_portfolio_valuation", metrics.get('Portfolio Valuation'))
                self.logger.record("episode/return_vs_market_pct", (portfolio_return - market_return) * 100)
                self.logger.record("episode/total_portfolio_return_pct", portfolio_return * 100)
                self.logger.record("episode/market_return_pct", market_return * 100)
                self.logger.record("episode/steps", raw_env.step) # CORRECTION : raw_env.step

                self.logger.dump(step=self.num_timesteps)

        return True

## Chargement et Configuration

In [26]:
# --- 1. HYPERPARAMÈTRES ET CONFIGURATION GLOBALE ---
config = {
    "policy_type": "MlpLstmPolicy",
    "total_timesteps": 100_000,
    "env_id": "MultiDatasetTradingEnv",
    "learning_rate": 3e-4,
    "n_steps": 2048,
    "batch_size": 128,
    "ent_coef": 0.01,
    "portfolio_initial_value": 1_000,
    "trading_fees": 0.1/100,
    "borrow_interest_rate": 0.02/100/24,
    "positions_range": (-1, 1),
    "model_name": "mon_agent_trading" # Nom de base pour la sauvegarde
}

# --- 2. INITIALISATION DE WANDB ET RÉCUPÉRATION DU CHEMIN DE SAUVEGARDE ---
run = wandb.init(
    project="RL-Trading-Project",
    entity="arthur-collignon-cpe-lyon",
    config=config,
    sync_tensorboard=True,
    monitor_gym=True,
    save_code=True,
)

# Chemin où SB3 sauvegardera le modèle (dans le dossier WandB)
# Nous stockons ce chemin dans une variable globale pour le backtesting.
MODEL_SAVE_PATH = f"models/{run.id}/{config['model_name']}.zip"
WANDB_RUN_ID = run.id # Sauvegarde de l'ID du run actuel

# --- 3. CRÉATION DE L'ENVIRONNEMENT ET DU MODÈLE ---
env = gym.make(
    config["env_id"],
    dataset_dir="data/*.pkl",
    preprocess=preprocess_v2,
    reward_function=reward_function_v2,
    position_range=config["positions_range"],
    portfolio_initial_value=config["portfolio_initial_value"],
    trading_fees=config["trading_fees"],
    borrow_interest_rate=config["borrow_interest_rate"],
)
env = DummyVecEnv([lambda: env])

model = RecurrentPPO(
    config["policy_type"], env, verbose=0, **{k: config[k] for k in ['learning_rate', 'n_steps', 'batch_size', 'ent_coef']}
)

# --- 4. DÉFINITION DE LA LISTE DE CALLBACKS ---
callback = CallbackList([
    WandbCallback(
        model_save_path=f"models/{run.id}",
        verbose=0,
        model_save_freq=10000,
        # Nom de fichier personnalisé (pour le rendre facilement chargeable)
        # Note: ceci nécessite un hack pour s'assurer que le nom est constant
        # La sauvegarde finale sera gérée manuellement.
    ),
    CustomTradingCallback(verbose=0),
])

# --- 5. ENTRAÎNEMENT ET SAUVEGARDE FINALE ---
try:
    print("Début de l'entraînement avec WandB...")
    model.learn(
        total_timesteps=config["total_timesteps"],
        callback=callback,
    )
    # Sauvegarde finale manuelle dans le chemin exact
    model.save(MODEL_SAVE_PATH)
    print(f"Modèle sauvegardé dans : {MODEL_SAVE_PATH}")
finally:
    run.finish()

Début de l'entraînement avec WandB...


wandb: WARNING Linked 1 file into the W&B run directory (hardlinks); call wandb.save again to sync new files.


Market Return :  5.54%   |   Portfolio Return : -99.99%   |   
Market Return : 278.54%   |   Portfolio Return : -100.00%   |   
Market Return : 43.52%   |   Portfolio Return : -65.81%   |   
Market Return : 103.41%   |   Portfolio Return : -93.95%   |   
Modèle sauvegardé dans : models/k9vxs8ed/mon_agent_trading.zip


## Exécution du Backtest (La boucle de prédiction)

In [28]:
# Nécessite matplotlib, numpy, pandas (déjà importés)

# --- RAPPEL DU CHEMIN DE SAUVEGARDE (doit être exécuté après la section 2) ---
# Si la section 2 a été exécutée, MODEL_SAVE_PATH et WANDB_RUN_ID sont définis.

# Si le notebook a été redémarré, vous devez retrouver le chemin du dernier run :
# Si vous voulez tester le dernier run, décommenter et exécuter cette recherche:
# latest_run_dir = os.path.join("wandb", "latest-run")
# path_to_load = os.path.join(latest_run_dir, "files", "models", "mon_agent_trading.zip")
# Sinon, utilisez le chemin stocké lors du run :
path_to_load = MODEL_SAVE_PATH # Assurez-vous que cette variable est définie !


# --- 1. Chargement de l'environnement de Test (Doit correspondre exactement à l'entraînement) ---
env_test = gym.make(
    config["env_id"],
    dataset_dir="data/*.pkl",
    preprocess=preprocess_v2,
    reward_function=reward_function_v2,
    position_range=config["positions_range"],
    portfolio_initial_value=config["portfolio_initial_value"],
    trading_fees=config["trading_fees"],
    borrow_interest_rate=config["borrow_interest_rate"],
)
env_test = DummyVecEnv([lambda: env_test])

# --- 2. Chargement de l'Agent depuis WandB ---
try:
    model = RecurrentPPO.load(path_to_load)
    print(f"Modèle chargé depuis : {path_to_load}")
except FileNotFoundError:
    print(f"ERREUR: Fichier modèle non trouvé à {path_to_load}.")
    print("Assurez-vous que l'entraînement s'est terminé et a bien sauvegardé le modèle.")
    # On arrête le backtest ici si le modèle n'est pas trouvé.
    exit()


# --- 3. Exécution du Backtest ---
obs, info = env_test.reset()
_states = None
done = False
while not done:
    # deterministic=True est CRUCIAL pour le test
    action, _states = model.predict(obs, state=_states, deterministic=True)
    obs, reward, terminated, truncated, info = env_test.step(action)
    done = terminated or truncated
print("Backtest terminé.")


# --- 4. Analyse des Résultats et Visualisation ---
raw_env = env_test.envs[0].unwrapped
metrics = raw_env.get_metrics()
df_results = raw_env.save_for_render(dir=f"render_logs_{WANDB_RUN_ID}")

print("\n--- RÉSULTATS DU BACKTEST ---")
print(f"Valeur Finale du Portefeuille    : {metrics['Portfolio Valuation']:.2f} $")
print(f"Rendement de l'Agent             : {metrics['Portfolio Return']}")
print(f"Rendement du Marché (Buy & Hold) : {metrics['Market Return']}")
print("-" * 30)

#
# Comparaison Graphique
plt.figure(figsize=(15, 6))
plt.plot(df_results.index, df_results['portfolio_valuation'], label='Agent AI', color='blue', linewidth=2)
# Buy & Hold (normalisé au capital initial)
initial_price = df_results['data_close'].iloc[0]
initial_portfolio = df_results['portfolio_valuation'].iloc[0]
factor = initial_portfolio / initial_price
plt.plot(df_results.index, df_results['data_close'] * factor, label='Buy & Hold (Marché)', color='gray', linestyle='--', alpha=0.7)
plt.title("Performance : Intelligence Artificielle vs Marché")
plt.xlabel("Temps")
plt.ylabel("Valeur du Portefeuille ($)")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


# Visualisation des Positions
plt.figure(figsize=(15, 4))
plt.plot(df_results.index, df_results['position'], label="Position de l'Agent", color='orange')
plt.title("Décisions de l'Agent au cours du temps")
plt.ylabel("Position (-1 = Short, 1 = Long)")
plt.show()

Modèle chargé depuis : models/k9vxs8ed/mon_agent_trading.zip


ValueError: not enough values to unpack (expected 2, got 1)